In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
import torch.nn as nn
from glob import glob
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from itertools import product
from torch.utils.data import TensorDataset, DataLoader
from astra.torch.models import EfficientNetClassifier,EfficientNet_B0_Weights
from astra.torch.utils import train_fn

import torchvision.models as models
from astra.torch.metrics import accuracy_score, f1_score, precision_score, recall_score,classification_report


NameError: name '_C' is not defined

In [ ]:
# s=0.5
# color_jitter = transforms.ColorJitter(0.4*4, 0.4*s, 0.4*s, 0.2*s)
# transform_eval = transforms.Compose([
#         # transforms.RandomResizedCrop((224,224), scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333)),
#         transforms.RandomHorizontalFlip(p=0.5),
#         transforms.RandomApply([color_jitter], p=0.2), 
#         transforms.RandomGrayscale(p=0.4),
#         # transforms.Resize(224),
#         # transforms.ToTensor(),
#         # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
#     ])

In [ ]:
loaded_data1 = torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/data.pt")
images= loaded_data1['images']
labels = loaded_data1['labels']
images = images / 255
images = (images - images.mean(dim=(0, 2, 3), keepdim=True)) / images.std(dim=(0, 2, 3), keepdim=True)

In [ ]:
train_images = images[:15500]
train_labels = labels[:15500]
test_images = images[15500:]
test_labels = labels[15500:]
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

In [ ]:
train_labels_count = (train_labels == 1).sum()
test_labels_count = (test_labels == 1).sum()

print("Number of train labels with value 1:", train_labels_count)
print("Number of test labels with value 1:", test_labels_count)

In [ ]:
# train_images=transform_eval(train_images)


train_set=torch.utils.data.TensorDataset(train_images,train_labels)
test_set=torch.utils.data.TensorDataset(test_images,test_labels)
train_loader=torch.utils.data.DataLoader(train_set,batch_size=512,shuffle=True,num_workers=8)
test_loader=torch.utils.data.DataLoader(test_set,batch_size=512,shuffle=False,num_workers=8)

In [ ]:
from astra.torch.models import EfficientNet,ResNetClassifier, MLP, MLPClassifier, EfficientNetClassifier

train_model = EfficientNetClassifier(
    models.efficientnet_b0,EfficientNet_B0_Weights, n_classes=2, activation=nn.ReLU(), dropout=0.1
)

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

class Simclr_astra(nn.Module):
    def __init__(self, model, n_classes, featurizer=None, projection_head=None, dropout=0.1,in_features=None):
        super(Simclr_astra, self).__init__()
        self.base_model = train_model
        self.n_features = in_features

        self.base_model.classifier.classifier.in_features= in_features
        self.base_model.classifier.classifier = nn.Linear(in_features, in_features)
        self.base_model.classifier.avtivation = nn.Identity()
        self.base_model.classifier.dropout = nn.Identity()
        self.base_model.classifier.classifier1 = nn.Identity()
        self.projectionHead = nn.Sequential(
            nn.Linear(self.n_features, self.n_features, bias=False),
            nn.BatchNorm1d(self.n_features),
            nn.ReLU(), #non-linearity
            nn.Linear(self.n_features, projection_head, bias=False),
            nn.BatchNorm1d(projection_head),
        )
    def forward(self, x_i, x_j):
        h_i = self.base_model(x_i)
        h_j = self.base_model(x_j)
        
        z_i = self.projectionHead(h_i)
        z_j = self.projectionHead(h_j)
        
        return h_i, h_j, z_i, z_j

In [ ]:
model=Simclr_astra(train_model, 2, featurizer=None, projection_head=128, dropout=0.1,in_features=512)

In [ ]:
model

In [ ]:
net=model
net

In [ ]:
print(model.base_model.classifier.classifier.weight)

In [ ]:
path_paramsend = "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/Saved_model/paramsend__simclr_astra_bk_DB_15000.pt"
checkpoint = torch.load(path_paramsend, map_location=torch.device('cpu'))
loss= checkpoint['loss']
epoch = checkpoint['epoch']
# start_epoch=0
epoch
# net.load_state_dict(checkpoint['net'])

In [ ]:
epoch

In [ ]:
print(model.base_model.classifier.classifier.weight)

In [ ]:
final_model = net.base_model
final_model.classifier.avtivation = nn.ReLU()  
final_model.classifier.dropout = nn.Dropout(0.1)
final_model.classifier.classifier1 = nn.Linear(512, 2)

In [ ]:
final_model 

In [ ]:
final_model=final_model.to('cuda')
iter_losses, epoch_losses = train_fn(
    final_model,
    nn.CrossEntropyLoss(),
    dataloader=train_loader,
    lr=3e-4,
    epochs=50,
    verbose=True,
    wandb_log=False,
)

In [ ]:
plt.plot(epoch_losses)

In [ ]:
with torch.no_grad():
    pred_classes =final_model.predict_class(
        dataloader=test_loader, batch_size=512, verbose=True
    ).to('cuda')

test_labels = test_labels.to('cuda')
print("Accuracy: ", accuracy_score(pred_classes,test_labels))
print("Precision: ", precision_score(pred_classes,test_labels))
print("Recall: ", recall_score(pred_classes,test_labels))
print("F1: ", f1_score(pred_classes,test_labels))
print("classification report: ", classification_report(pred_classes,test_labels))